## LLM

In [1]:
import matplotlib.pyplot as plt
import random
from IPython.display import clear_output
from src.tokenizer import TokenizerBPE, word_split, normalize_to_ascii, pair_freq
from src.data_handling import read_first_n


import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf
import numpy as np
import pickle as pkl
from tqdm.notebook import tqdm
from src.data_handling import fused_qa
import sys
import random

# disable gpu for testing purposes
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [2]:
import json


with open('corpus/train-v2.0.json', 'r', encoding='utf-8') as f:
    squad = json.load(f)

In [3]:
question_list = []
answer_list = []

for article in squad['data']:
    title = article['title']
    print(f"Title: {title}")
    for paragraph in article['paragraphs']:
        for qa in paragraph['qas']:
            if not qa['is_impossible']:
                question = qa['question']
                answer = qa['answers']
                if title == "The_Legend_of_Zelda:_Twilight_Princess":
                    print(f"Question: {question}")
                    print(f"Answer: {answer[0]['text']}")

                question_list.append(question.lower())
                answer_list.append(answer[0]['text'].lower())

question_list.pop(71681)
answer_list.pop(71681)             

Title: Beyoncé
Title: Frédéric_Chopin
Title: Sino-Tibetan_relations_during_the_Ming_dynasty
Title: IPod
Title: The_Legend_of_Zelda:_Twilight_Princess
Question: What category of game is Legend of Zelda: Twilight Princess?
Answer: action-adventure
Question: What consoles can be used to play Twilight Princess?
Answer: GameCube and Wii
Question: When was Twilight Princess launched in North America?
Answer: November 2006
Question: When could GameCube owners purchase Twilight Princess?
Answer: December 2006
Question: What company developed Legend of Zelda: Twilight Princess?
Answer: Nintendo
Question: What year was the Legend of Zelda:Twilight Princess originally planned for release?
Answer: 2005
Question: What year was the Wii version of Legend of Zelda: Twilight Princess released?
Answer: 2006
Question: Who is the main character of the story?
Answer: Link
Question: What land does Link serve to protect?
Answer: Hyrule
Question: What character helped Link in Twilight Princess?
Answer: Midna


'horizontal dipole'

In [4]:
corpus_qa = question_list + answer_list
corpus_qa_str = "".join(corpus_qa)
print(len(corpus_qa_str))

6898856


In [5]:
question_list[0]

'when did beyonce start becoming popular?'

In [6]:
random.seed(42)
corpus_cnn = read_first_n('corpus/stories', 90000)
random.shuffle(corpus_cnn)
corpus_cnn = corpus_cnn[:3000]
corpus_cnn = [line.lower() for line in corpus_cnn]

In [8]:
corpus_cnn_str = "".join(corpus_cnn)
print(len(corpus_cnn_str))

12768817


In [ ]:
random.seed(43)

corpus = question_list + answer_list + corpus_cnn

tokenizer = TokenizerBPE(corpus, 8000, lowercase=True)
tokenizer.destroy_hash()


pkl.dump(tokenizer, open('tokenizers/CNN_QA8k_balanced.pkl', 'wb'))

Lowercasing corpus


  0%|          | 0/176640 [00:00<?, ?it/s]

Merging tokens


  0%|          | 0/8000 [00:00<?, ?it/s]

t h 368040
i n 296426
th e 252886
a n 237709
e r 224647
a t 211507
o n 205477
r e 186987
e n 160406
s t 141668
o r 141314
e d 135475
a l 128685
a r 118515
w h 109587
o f 108105
i s 107614
t o 105746
a s 103292
e s 102253
i t 101253
o u 97587
in g 96571
i c 91378
an d 84840
i on 72854
i d 70609
l e 70576
o m 66039
en t 59632
c h 58421
i l 56345
i g 55826
wh at 55085
s e 54540
b e 49890
a m 47981
a c 47145
r o 46912
v e 45181
a d 44411
f or 42993
o l 41830
o w 38764
a y 38442
ig h 37489
h e 36159
' s 36067
w as 34921
u n 34277
d e 34244
u r 33806
l y 33629
i m 33290
i r 32122
at ion 32077
t er 30748
v er 30703
c on 29478
th at 28946
p e 28643
c e 28498
g e 28370
u s 28333
o t 27688
n e 27296
a re 26968
o p 25982
u t 25652
t r 24696
a b 24118
s a 23350
c t 23335
w e 23283
m o 23098
c om 23093
l i 23080
f r 22907
w i 22664
re s 22443
h a 22096
igh t 21798
u l 21511
wi th 21055
t e 21040
l d 20606
ou n 20431
m an 20357
at e 20292
il l 20150
er s 20072
er e 19633
i st 19380
m ent 19232
al l 

In [9]:
def fused_qa(question_list, answer_list):
    q ="<q>"
    a = "<a>"
    sos = "<s>"
    eos = "</s>"

    corpus_list = []
    for question, answer in tqdm(list(zip(question_list, answer_list))):
        qa = [sos, q, question, a, answer, eos]
        corpus_list.extend(qa)
    
    return "".join(corpus_list)


def fused_cnn(corpus_list):
    sos = "<s>"
    eos = "</s>"

    corpus_padded_list = []
    for line in corpus_list:
        line = [sos, line, eos]
        corpus_padded_list.extend(line)
    
    return "".join(corpus_padded_list)

In [10]:
tokenizer = pkl.load(open('tokenizers/CNN_QA8k_balanced.pkl', 'rb'))
tokenizer.create_hash()

tokenizer.add_special_tokens(["<s>", "</s>", "<q>", "<a>",])

In [11]:
question_list = [line.lower() for line in question_list]
answer_list = [line.lower() for line in answer_list]

In [12]:
corpus_qa_fused = fused_qa(question_list, answer_list)

corpus_qa_encoded = tokenizer.encode(corpus_qa_fused)

  0%|          | 0/86820 [00:00<?, ?it/s]

  0%|          | 0/8000 [00:00<?, ?it/s]

In [13]:
corpus_cnn_fused = fused_cnn(corpus_cnn)

corpus_cnn_encoded = tokenizer.encode(corpus_cnn_fused)

  0%|          | 0/8000 [00:00<?, ?it/s]

In [14]:
corpus = tf.concat([corpus_cnn_encoded, corpus_qa_encoded], axis=1)
pkl.dump(corpus, open('corpus/corpus_CNN_QA8k_balanced', 'wb'))

## 